In [1]:
from tiatoolbox.models import MapDe
from tiatoolbox.utils.misc import select_device
from tiatoolbox.wsicore.wsireader import WSIReader
import torch
import numpy as np
import matplotlib.pyplot as plt
from tiatoolbox.tools.patchextraction import get_patch_extractor
from tiatoolbox.annotation.storage import Annotation, SQLiteStore
from shapely import Point, Polygon, box

# print(fetch_pretrained_weights("mapde-conic"))

|2024-10-23|16:35:15.790| [WARNING] /home/u1910100/miniconda3/envs/tiatoolbox/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/u1910100/miniconda3/envs/tiatoolbox/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(

|2024-10-23|16:35:17.228| [WARNING] /home/u1910100/miniconda3/envs/tiatoolbox/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecat

In [7]:
def load_mapde() -> MapDe:
    """Loads MapDe model with specified weights."""
    model = MapDe(min_distance=4, threshold_abs=250)
    weights_path = (
        "/home/u1910100/GitHub/Monkey_TIAKong/mapde-conic.pth"
    )
    map_location = select_device(on_gpu=True)
    pretrained = torch.load(weights_path, map_location=map_location)
    model.load_state_dict(pretrained)

    return model

In [3]:
def points_to_annotation_store(points: list, scale_factor: float = 1):
    """
    Args: points(list): list of (x,y) coordinates
    """
    annotation_store = SQLiteStore()

    for coord in points:
        x = int(coord[0] * scale_factor)
        y = int(coord[1] * scale_factor)
        annotation_store.append(
            Annotation(
                geometry=Polygon.from_bounds(
                    x - 16, y - 16, x + 16, y + 16
                ),
                properties={"class": 1, "type": "Cell"},
            )
            # Annotation(geometry=Point(coord[0], coord[1]), properties={"class": 1, "type": "TIL"})
        )

    return annotation_store


def scale_coords(coords: list, scale_factor: float = 1):
    new_coords = []
    for coord in coords:
        x = int(coord[0] * scale_factor)
        y = int(coord[1] * scale_factor)
        new_coords.append([x, y])

    return new_coords

In [12]:
wsi_path = "/home/u1910100/Documents/Monkey/test/input/images/kidney-transplant-biopsy-wsi-pas/A_P000001_PAS_CPG.tif"
mask_path = "/home/u1910100/Documents/Monkey/test/input/images/tissue-mask/A_P000001_mask.tif"

wsi_reader = WSIReader.open(wsi_path)


print(
    wsi_reader.convert_resolution_units(
        input_res=0, input_unit="level", output_unit="mpp"
    )
)


mask_reader = WSIReader.open(mask_path)
mask_thumbnail = mask_reader.slide_thumbnail()
binary_mask = mask_thumbnail[:, :, 0]

patch_extractor = get_patch_extractor(
    input_img=wsi_reader,
    input_mask=binary_mask,
    method_name="slidingwindow",
    patch_size=(252, 252),
    stride=(252, 252),
    resolution=0.5,
    units="mpp",
)

print(len(patch_extractor))
mapde_result = []

model = load_mapde()
model.to("cuda")

for i, patch in enumerate(patch_extractor):
    bb = patch_extractor.coordinate_list[i]

    patch_input = torch.from_numpy(patch)[None]
    patch_input.to("cuda").float()

    output = model.infer_batch(model, patch_input, on_gpu=True)
    output = model.postproc(output[0])
    for coord in output:
        mapde_result.append([coord[0] + bb[0], coord[1] + bb[1]])

|2024-10-23|16:43:55.433| [WARNING] Metadata: Falling back to TIFF resolution tag for microns-per-pixel (MPP).
|2024-10-23|16:43:55.434| [WARNING] Metadata: Objective power inferred from microns-per-pixel (MPP).
|2024-10-23|16:43:55.437| [WARNING] Metadata: Falling back to TIFF resolution tag for microns-per-pixel (MPP).
|2024-10-23|16:43:55.438| [WARNING] Metadata: Objective power inferred from microns-per-pixel (MPP).


[0.24199951 0.24199951]
56


|2024-10-23|16:43:56.646| [WARNING] /home/u1910100/miniconda3/envs/tiatoolbox/lib/python3.11/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)



In [9]:
scale_factor = 0.5 / 0.24199951445730394
scaled_result = scale_coords(mapde_result, scale_factor)
print(scaled_result)

annoation_store = points_to_annotation_store(scaled_result)

annoation_store.dump(
    "/home/u1910100/Documents/Monkey/test/input/images/overlays/A_P000001_PAS_CPG.db"
)

[[10795, 86407], [10731, 86409], [11097, 86351], [11194, 86388], [11274, 86225], [11247, 86301], [11836, 86407], [11971, 86376], [12152, 86411], [12400, 86413], [12477, 86409], [12803, 86405], [10305, 86890], [10677, 86929], [10611, 86686], [10557, 86508], [11173, 86597], [11006, 86748], [11440, 86434], [10989, 86642], [11035, 86601], [13531, 86812], [13130, 86754], [13053, 86595], [13448, 86580], [13528, 86589], [13706, 86936], [14053, 86851], [10305, 87411], [9900, 87403], [10165, 87403], [9952, 87452], [10562, 87442], [11028, 87438], [11929, 87378], [11966, 87093], [11969, 87266], [11923, 86952], [12008, 87372], [12092, 87093], [13642, 86952], [13785, 87235], [10002, 87836], [10214, 87944], [9933, 87942], [10074, 87839], [10927, 87653], [10747, 87572], [10623, 87913], [11270, 87969], [11318, 87679], [11927, 87634], [11915, 87762], [11958, 87787], [11894, 87632], [11619, 87973], [12049, 87560], [12995, 87733], [10014, 88452], [10099, 88289], [10396, 88357], [11272, 88239], [11520, 88

In [11]:
from monkey.data.data_utils import px_to_mm, write_json_file
import os


output_dict_lymphocytes = {
    "name": "lymphocytes",
    "type": "Multiple points",
    "version": {"major": 1, "minor": 0},
    "points": [],
}

output_dict_monocytes = {
    "name": "monocytes",
    "type": "Multiple points",
    "version": {"major": 1, "minor": 0},
    "points": [],
}

output_dict_inflammatory_cells = {
    "name": "inflammatory-cells",
    "type": "Multiple points",
    "version": {"major": 1, "minor": 0},
    "points": [],
}

for i, coord in enumerate(scaled_result):
    counter = i + 1
    x = coord[0]
    y = coord[1]
    confidence = 0.9
    prediction_record = {
        "name": "Point " + str(counter),
        "point": [
            px_to_mm(x, 0.24199951445730394),
            px_to_mm(y, 0.24199951445730394),
            0.24199951445730394,
        ],
        "probability": confidence,
    }
    output_dict_lymphocytes["points"].append(prediction_record)
    output_dict_monocytes["points"].append(
        prediction_record
    )  # should be replaced with detected monocytes
    output_dict_inflammatory_cells["points"].append(prediction_record)


json_filename = "detected-lymphocytes.json"
output_path = "/home/u1910100/Documents/Monkey/test/output"
output_path_json = os.path.join(output_path, json_filename)
write_json_file(
    location=output_path_json, content=output_dict_lymphocytes
)

json_filename_monocytes = "detected-monocytes.json"
# it should be replaced with correct json files
output_path_json = os.path.join(output_path, json_filename_monocytes)
write_json_file(
    location=output_path_json, content=output_dict_monocytes
)

json_filename_inflammatory_cells = "detected-inflammatory-cells.json"
# it should be replaced with correct json files
output_path_json = os.path.join(
    output_path, json_filename_inflammatory_cells
)
write_json_file(
    location=output_path_json, content=output_dict_inflammatory_cells
)